In [8]:
import pandas as pd  # Data handling
import matplotlib.pyplot as plt  # Basic plotting
import seaborn as sns  # Advanced visualizations
import datetime as dt
import ipywidgets as widgets


In [28]:
import pandas as pd

def load_data():
    try:
        df = pd.read_csv("data.csv")
        # Ensure the 'Date' column is in the correct format after loading
        df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y", errors="coerce").dt.strftime("%d-%m-%Y")
    except FileNotFoundError:
        df = pd.DataFrame(columns=['Date', 'Category', 'Amount', 'Description'])
    return df

def save_data(df):
    df.to_csv("data.csv", index=False)

def add_expense(Date, Category, Amount, Description):
    df = load_data()
    
    new_row = pd.DataFrame({
        'Date': [Date.strftime("%d-%m-%Y")],  # Always save as string in dd-mm-yyyy format
        'Category': [Category],
        'Amount': [Amount],
        'Description': [Description]
    })
    
    df = pd.concat([df, new_row], ignore_index=True)
    
    # Make sure the 'Date' column is consistent
    df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y", errors="coerce").dt.strftime("%d-%m-%Y")
    
    # Add day, month, and year columns
    df["Day"] = pd.to_datetime(df["Date"], format="%d-%m-%Y").dt.day
    df["Month"] = pd.to_datetime(df["Date"], format="%d-%m-%Y").dt.month
    df["Year"] = pd.to_datetime(df["Date"], format="%d-%m-%Y").dt.year
    
    save_data(df)
    return df

def summary_view(df, period="day"):
    df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y", dayfirst=True)  # Ensure date is in datetime format
    df["Amount"] = pd.to_numeric(df["Amount"], errors='coerce')  # Ensure Amount is numeric

    if period == "day":
        summary = df.groupby(df["Date"])["Amount"].sum()
    elif period == "week":
        df["Week"] = df["Date"].dt.to_period("W").astype(str)
        summary = df.groupby("Week")["Amount"].sum()
    elif period == "month":
        df["Month"] = df["Date"].dt.to_period("M").astype(str)
        summary = df.groupby("Month")["Amount"].sum()
    else:
        print("Invalid period. Choose 'day', 'week', or 'month'.")
        return

    print(f"\nTotal spending by {period.capitalize()}:\n", summary)

# Input Expense
while True:
    Date = input("Enter date (dd-mm-yyyy): ").strip()
    try:
        Date = pd.to_datetime(Date, format="%d-%m-%Y", dayfirst=True)
        print("Date is valid!")
        break
    except ValueError:
        print("Invalid date format. Please enter dd-mm-yyyy.")

Category = input("Category (Food, Transport, etc.): ").strip()

while True:
    try:
        Amount = float(input("How much? "))
        break
    except ValueError:
        print("Invalid input. Please enter a valid number.")

Description = input("Description (Optional): ").strip()
if not Description:
    Description = "No description provided"

# Add Expense and Print Updated DataFrame
df = add_expense(Date, Category, Amount, Description)
print(df)


Enter date (dd-mm-yyyy): 20-02-2025
Date is valid!
Category (Food, Transport, etc.): Food
How much? 400
Description (Optional): 
         Date   Category  Amount              Description  Day  Month  Year
0  20-02-2025       Food    40.0  No description provided   20      2  2025
1  28-02-2025  Transport    80.0  No description provided   28      2  2025
2  20-02-2025       Food   400.0  No description provided   20      2  2025


In [29]:
summary_view(df, "day")   # Daily total
summary_view(df, "week")  # Weekly total
summary_view(df, "month") # Monthly total


Total spending by Day:
 Date
2025-02-20    440.0
2025-02-28     80.0
Name: Amount, dtype: float64

Total spending by Week:
 Week
2025-02-17/2025-02-23    440.0
2025-02-24/2025-03-02     80.0
Name: Amount, dtype: float64

Total spending by Month:
 Month
2025-02    520.0
Name: Amount, dtype: float64
